In [20]:
# https://github.com/bayartsogt-ya/whisper-multiple-hf-datasets/blob/main/src/multiple_datasets/hub_default_utils.py

from copy import deepcopy
import torch
from transformers import WhisperForConditionalGeneration


WHISPER_MAPPING = {
    "layers": "blocks",
    "fc1": "mlp.0",
    "fc2": "mlp.2",
    "final_layer_norm": "mlp_ln",
    "layers": "blocks",
    ".self_attn.q_proj": ".attn.query",
    ".self_attn.k_proj": ".attn.key",
    ".self_attn.v_proj": ".attn.value",
    ".self_attn_layer_norm": ".attn_ln",
    ".self_attn.out_proj": ".attn.out",
    ".encoder_attn.q_proj": ".cross_attn.query",
    ".encoder_attn.k_proj": ".cross_attn.key",
    ".encoder_attn.v_proj": ".cross_attn.value",
    ".encoder_attn_layer_norm": ".cross_attn_ln",
    ".encoder_attn.out_proj": ".cross_attn.out",
    "decoder.layer_norm.": "decoder.ln.",
    "encoder.layer_norm.": "encoder.ln_post.",
    "embed_tokens": "token_embedding",
    "encoder.embed_positions.weight": "encoder.positional_embedding",
    "decoder.embed_positions.weight": "decoder.positional_embedding",
    "layer_norm": "ln_post",
}


def rename_keys(s_dict):
    keys = list(s_dict.keys())
    for key in keys:
        new_key = key
        for k, v in WHISPER_MAPPING.items():
            if k in key:
                new_key = new_key.replace(k, v)

        print(f"{key} -> {new_key}")

        s_dict[new_key] = s_dict.pop(key)
    return s_dict


def convert_hf_whisper(hf_model_name_or_path: str, whisper_state_path: str):
    transformer_model = WhisperForConditionalGeneration.from_pretrained(hf_model_name_or_path)
    config = transformer_model.config

    # first build dims
    dims = {
        'n_mels': config.num_mel_bins,
        'n_vocab': config.vocab_size,
        'n_audio_ctx': config.max_source_positions,
        'n_audio_state': config.d_model,
        'n_audio_head': config.encoder_attention_heads,
        'n_audio_layer': config.encoder_layers,
        'n_text_ctx': config.max_target_positions,
        'n_text_state': config.d_model,
        'n_text_head': config.decoder_attention_heads,
        'n_text_layer': config.decoder_layers
    }

    state_dict = deepcopy(transformer_model.model.state_dict())
    state_dict = rename_keys(state_dict)

    torch.save({"dims": dims, "model_state_dict": state_dict}, whisper_state_path)

In [21]:
convert_hf_whisper('mesolitica/finetune-whisper-small-ms-singlish-v2', 'model.pt')

encoder.conv1.weight -> encoder.conv1.weight
encoder.conv1.bias -> encoder.conv1.bias
encoder.conv2.weight -> encoder.conv2.weight
encoder.conv2.bias -> encoder.conv2.bias
encoder.embed_positions.weight -> encoder.positional_embedding
encoder.layers.0.self_attn.k_proj.weight -> encoder.blocks.0.attn.key.weight
encoder.layers.0.self_attn.v_proj.weight -> encoder.blocks.0.attn.value.weight
encoder.layers.0.self_attn.v_proj.bias -> encoder.blocks.0.attn.value.bias
encoder.layers.0.self_attn.q_proj.weight -> encoder.blocks.0.attn.query.weight
encoder.layers.0.self_attn.q_proj.bias -> encoder.blocks.0.attn.query.bias
encoder.layers.0.self_attn.out_proj.weight -> encoder.blocks.0.attn.out.weight
encoder.layers.0.self_attn.out_proj.bias -> encoder.blocks.0.attn.out.bias
encoder.layers.0.self_attn_layer_norm.weight -> encoder.blocks.0.attn_ln.weight
encoder.layers.0.self_attn_layer_norm.bias -> encoder.blocks.0.attn_ln.bias
encoder.layers.0.fc1.weight -> encoder.blocks.0.mlp.0.weight
encoder.l

In [23]:
import whisper

model = whisper.load_model('model.pt')

In [24]:
audio = whisper.load_audio("husein-zolkepli.wav")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Detected language: en
nama saya hussein bin zulkifli


In [25]:
from malaya_boilerplate.huggingface import upload_dict

In [26]:
files_mapping = {'model.pt': 'model.pt'}
upload_dict(model = 'finetune-whisper-small-ms-singlish-v2', 
            files_mapping = files_mapping, username = 'mesolitica')

/home/husein/.local/lib/python3.8/site-packages/huggingface_hub/hf_api.py:101: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.10. Pass `repo_id` instead.
  warnings.warn(
<class 'requests.exceptions.HTTPError'> (Request ID: Root=1-63d77042-1b1c3cab62a08dc360a12830)

You already created this model repo - You already created this model repo
